The following packages are used:
pyowm: a library for OpenWeatherMap (OWM) web API
folium: python wrapper for leaflet.js
json & request: an alternative way to access OpenWeatherMap API
city: A custom module to create city objects with: name, isocode, latitude, longitude and cloud coverage attributes.

In [1]:
import sys, pyowm, random, folium, json, requests
from city import City
from osgeo import gdal,ogr
from folium import plugins

In [2]:
# OpenWeather API Token & Connection
token = 'bde4245ffd54f082f64af5956459af48'
owm = pyowm.OWM(token)

# Load Cities shapefile
driver = ogr.GetDriverByName('ESRI Shapefile')
cityds = driver.Open("./cities/cities.shp")
if cityds is None:
    print ("Open failed.\n")
    sys.exit( 1 )
else:
    print("File opened")

File opened


In [3]:
# Get Shapefile Fields and Rows
layer = cityds.GetLayer()
featureCount = layer.GetFeatureCount()
layerDefinition = layer.GetLayerDefn()
fieldCount = layerDefinition.GetFieldCount()
print("Layer has {} fields and {} features".format(fieldCount, featureCount))


Layer has 14 fields and 7343 features


In [4]:
# Function to retrieve cloud cover from OpenWeatherAPI using the pyowm package
def cloudcover(name, isocode, latitude, longitude):
        """Retrieves weather data from Openweather API and returns a City object.
        
        Args:
            name(str): Name of the city.
            iscode(str): The country ISO alpha-2 code
            latitude(float): Latitude in degrees
            longitude(float): Longitude in degrees
        """
        obs = owm.weather_at_place(name + ',' + isocode)
        w = obs.get_weather()
        c = w.get_clouds()
        print("{} City has {}% cloud cover, with {}".format(name,c,w.get_detailed_status())) 
        return City(name, isocode, latitude, longitude, c)

In [5]:
# OPTION 2: Retrieving Cloud cover via JSON
def cloudcoverjson(name, isocode, latitude, longitude, token):
    url = ('http://api.openweathermap.org/data/2.5/weather?q='+name+'&APPID='+token)
    response = requests.get(url)
    data = response.json()
    c = data["clouds"]["all"]
    print("{} City has {}% cloud cover".format(name,c))
    return City(name, isocode, latitude, longitude, c)

Below, a city list is created by looping through the layer randomly to retrieve the cloud cover and create a city object ready for mapping. The city list can be restricted to a continent otherwise, 50 cities will be selected worldwide.

In [6]:
# Randomly select cities and get cloud coverage for 50 of them
continent = 'Europe'
layer_subset = random.choices(layer, k=7000)
citylist = []

for feature in layer_subset:
    name = feature.GetField("NAMEASCII")
    isocode = feature.GetField("ISO_A2")
    lat =  feature.GetField("LATITUDE")
    lon =  feature.GetField("LONGITUDE")
    if continent is None:
        try:
            city = cloudcover(name, isocode, lat, lon)
            citylist.append(city)
        except:
            pass
        if len(citylist) == 50:
            break
    else:
        if feature.GetField("CONTINENT")==continent:
            try:
                city = cloudcover(name,isocode,lat,lon)
                citylist.append(city)
            except:
                pass

        if len(citylist) == 50:
            break

Daloa City has 97% cloud cover, with overcast clouds
Nampula City has 75% cloud cover, with broken clouds
Guiglo City has 100% cloud cover, with moderate rain
Bulawayo City has 0% cloud cover, with clear sky
Ghadamis City has 0% cloud cover, with clear sky
Jendouba City has 40% cloud cover, with scattered clouds
Sumbe City has 0% cloud cover, with clear sky
Masasi City has 0% cloud cover, with clear sky
Biltine City has 4% cloud cover, with clear sky
Cradock City has 86% cloud cover, with overcast clouds
Libenge City has 100% cloud cover, with overcast clouds
Mochudi City has 0% cloud cover, with clear sky
Atbara City has 0% cloud cover, with clear sky
Uitenhage City has 40% cloud cover, with scattered clouds
Kosti City has 86% cloud cover, with overcast clouds
Bouar City has 93% cloud cover, with light rain
Kasempa City has 39% cloud cover, with scattered clouds
Mossel Bay City has 100% cloud cover, with overcast clouds
Mariental City has 0% cloud cover, with clear sky
Brazzaville Cit

In [7]:
# Position and zoom map wrt to continent selection
if continent == 'Africa':
    map = folium.Map(location=[0, 22], tiles='CartoDB positron', zoom_start=3)
elif continent == 'Asia':
    map = folium.Map(location=[31, 73], tiles='CartoDB positron', zoom_start=4)
elif continent == 'America':
    map = folium.Map(location=[7, -72], tiles='CartoDB positron', zoom_start=3)
elif continent == 'Europe':
    map = folium.Map(location=[47, 17], tiles='CartoDB positron', zoom_start=4)
else:
    map = folium.Map(location=[0, 0], tiles='CartoDB positron', zoom_start=2)

# Add Map Controls
plugins.Fullscreen(position='topleft', title='Fullscreen', title_cancel='Exit',force_separate_button=False).add_to(map)
folium.TileLayer('openstreetmap').add_to(map)
folium.LayerControl().add_to(map)

# Add city markers to the map
for city in citylist:
    name = city.get_name()
    lat = city.get_lat()
    lon = city.get_lon()
    cloud = city.get_cloudcover()

    if 0 < cloud <= 20:
        icon=folium.Icon(color='orange')
    elif cloud > 20:
        icon=folium.Icon(icon='cloud', color='red')
    else:
        icon=folium.Icon(color='green')
        
    folium.Marker([lat, lon], 
                  popup=name + ": " +str(cloud)+"%",
                  tooltip = "Cloud Coverage",
                  icon=icon).add_to(map)
# Add legend
legend =   '''
                <div style="position: fixed; 
                            bottom: 10px; left: 10px; width: 120px; height: 100px; 
                            border:0px solid grey; z-index:9999; font-size:12px;
                            background-color:rgb(245, 245, 245);
                            ">&nbsp; <b>% Cloud Coverage </b><br>
                              &nbsp; <i class="fa fa-map-marker fa-2x" style="color:green"></i> &nbsp; 0 <br>
                              &nbsp; <i class="fa fa-map-marker fa-2x" style="color:orange"></i> &nbsp; 1- 20 <br>
                              &nbsp; <i class="fa fa-map-marker fa-2x" style="color:red"></i> &nbsp; > 21
                </div>
                ''' 
map.get_root().html.add_child(folium.Element(legend))
map

In [8]:
cityds = None